In [6]:
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO
from sklearn.impute import SimpleImputer
import joblib
import os

In [7]:
# Load trained classifier
model = joblib.load("exercise_classifier.pkl")

# Define class labels
class_map = {0: "Pushup", 1: "Squat", 2: "Pullup"}

# Load YOLO pose model
pose_model = YOLO("yolo11n-pose.pt")

In [8]:
# Load video
video_path = "videos/snapshots/Snapshot_video1.mov"
#video_path = "videos/single_pushup_videos/mirrored_single_pushup_random2.mov"

cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frames = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(frame)
cap.release()

if len(frames) < 6:
    print("⚠️ Not enough frames.")
    exit()

# === Select 6 equally spaced frames from the video ===
sample_idxs = np.linspace(0, len(frames) - 1, min(10, len(frames)), dtype=int)
sample_idxs = np.unique(sample_idxs)

In [9]:
# === Feature functions ===
def angle_between_points(p1, p2, p3):
    a = np.array([p1[0] - p2[0], p1[1] - p2[1]])
    b = np.array([p3[0] - p2[0], p3[1] - p2[1]])
    if np.any(np.isnan(a)) or np.any(np.isnan(b)) or np.linalg.norm(a) == 0 or np.linalg.norm(b) == 0:
        return np.nan
    cos_angle = np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
    return np.degrees(np.arccos(np.clip(cos_angle, -1.0, 1.0)))

def euclidean_distance(p1, p2):
    if np.any(np.isnan(p1)) or np.any(np.isnan(p2)):
        return np.nan
    return np.linalg.norm(np.array(p1) - np.array(p2))

def compute_joint_features(df):
    features = []
    for _, row in df.iterrows():
        kp = lambda i: (row[f"kp_{i}_x"], row[f"kp_{i}_y"])  # works with named columns
        feat = {
            "feat_elbow_angle_L": angle_between_points(kp(5), kp(7), kp(9)),
            "feat_elbow_angle_R": angle_between_points(kp(6), kp(8), kp(10)),
            "feat_knee_angle_L": angle_between_points(kp(11), kp(13), kp(15)),
            "feat_knee_angle_R": angle_between_points(kp(12), kp(14), kp(16)),
            "feat_hip_angle_L": angle_between_points(kp(5), kp(11), kp(13)),
            "feat_hip_angle_R": angle_between_points(kp(6), kp(12), kp(14)),
            "feat_shoulder_width": euclidean_distance(kp(5), kp(6)),
            "feat_hip_to_wrist_L": euclidean_distance(kp(11), kp(9)),
            "feat_hip_to_wrist_R": euclidean_distance(kp(12), kp(10)),
            "feat_knee_to_ankle_L": euclidean_distance(kp(13), kp(15)),
            "feat_knee_to_ankle_R": euclidean_distance(kp(14), kp(16)),
            "feat_hip_y": row["kp_11_y"],
        }
        features.append(feat)
    return pd.DataFrame(features)


In [10]:
# === Extract keypoints + engineered features ===
keypoint_data = []
feature_data = []

for i, idx in enumerate(sample_idxs):
    results = pose_model.predict(source=frames[idx], conf=0.25, save=False, verbose=False)
    keypoints_flat = [np.nan] * (17 * 3)

    try:
        result = results[0]
        if result.keypoints is not None:
            keypoints = result.keypoints.xy[0].cpu().numpy()
            confidences = result.keypoints.conf[0].cpu().numpy()

            flat = []
            for j in range(17):
                x = y = c = np.nan
                if j < len(confidences) and confidences[j] >= 0.2:
                    x, y = keypoints[j]
                    c = confidences[j]
                flat.extend([x, y, c])
            keypoints_flat = flat
    except Exception as e:
        print(f"⚠️ Pose estimation failed at frame {idx}: {e}")

    xy_headers = [f"kp_{k}_x_f{i}" for k in range(17)] + [f"kp_{k}_y_f{i}" for k in range(17)]
    conf_headers = [f"kp_{k}_conf_f{i}" for k in range(17)]
    df_kp = pd.DataFrame([keypoints_flat], columns=xy_headers + conf_headers)

    # Prepare for feature calculation
    df_simple = df_kp.copy()
    df_simple.columns = [col.replace(f"_f{i}", "") for col in df_kp.columns]
    df_feat = compute_joint_features(df_simple)
    df_feat.columns = [f"{col}_f{i}" for col in df_feat.columns]

    keypoint_data.append(df_kp[xy_headers])  # only x/y
    feature_data.append(df_feat)

# === Merge features from all frames
df_all_keypoints = pd.concat(keypoint_data, axis=1)
df_all_features = pd.concat(feature_data, axis=1)
combined_all = pd.concat([df_all_keypoints, df_all_features], axis=1)

# === Load expected columns from training ===
if not os.path.exists("used_feature_columns.txt"):
    print("❌ Missing file: used_feature_columns.txt. Run training first.")
    exit()

with open("used_feature_columns.txt", "r") as f:
    expected_columns = [line.strip() for line in f.readlines()]

# === Add missing columns and reorder ===
for col in expected_columns:
    if col not in combined_all.columns:
        combined_all[col] = np.nan
combined_all = combined_all[expected_columns]

# === Impute missing values ===
imputer = SimpleImputer(strategy="constant", fill_value=0)
X_sample = pd.DataFrame(imputer.fit_transform(combined_all), columns=expected_columns)

# === Final shape check ===
if X_sample.shape[1] != model.n_features_in_:
    print(f"❌ Feature mismatch: model expects {model.n_features_in_}, but got {X_sample.shape[1]}")
    exit()

# === Prediction ===
proba = model.predict_proba(X_sample)[0]
predicted_class = np.argmax(proba)
confidence = round(100 * proba[predicted_class])

# === Output result ===
print("\n🔍 Prediction Confidence per Class:")
for i, cls in class_map.items():
    print(f" - {cls}: {round(100 * proba[i])}%")

if confidence >= 40:
    print(f"\n✅ Final Prediction: {class_map[predicted_class]} ({confidence}%)")
else:
    print(f"\n⚠️ No defined exercise detected")



🔍 Prediction Confidence per Class:
 - Pushup: 44%
 - Squat: 32%
 - Pullup: 24%

✅ Final Prediction: Pushup (44%)
